# <center>New Unseen Data Prediction Regression</center>
---

<center>Malik Alrasyid Basori</center>

In [4]:
# Run locally & on the cloud
import joblib
import pandas as pd

# Variable Need for Google Cloud (or You could just upload all of them to Github)

In [ ]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [ ]:
## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'malik_dataset_034'
table_id = 'new_entries_in_california_housing'
region = 'us-central1'
bucket_name = 'malik_gcs_034'
blob_name = 'data/new_entries_in_california_housing.csv'

model_name = 'California_Housing_Price.pkl'

## Google Cloud Storage

### Upload to Google Cloud Storage - skipped

### Retrieve from Google Cloud Storage

In [ ]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('California_Housing_Price.pkl')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

## Bigquery

### Load from Bigquery

In [ ]:
#load data from BQ
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
batch_test = query_job.result().to_dataframe()

In [ ]:
batch_test.head()

# Model Prediction

## Cloud Data Prediction

In [ ]:
pip install category_encoders

In [2]:
# Function to preprocess new data
def preprocess_new_data(new_data):
    # Encode 'ocean_proximity'
    new_data['ocean_proximity'] = new_data['ocean_proximity'].replace({'ISLAND': 0, 'NEAR OCEAN': 1, '<1H OCEAN': 2, 'NEAR BAY': 3, 'INLAND': 4})
    
    # Create new features
    new_data['rooms_per_household'] = new_data['total_rooms'] / new_data['households']
    new_data['bedrooms_per_room'] = new_data['total_bedrooms'] / new_data['total_rooms']
    new_data['population_per_household'] = new_data['population'] / new_data['households']
    
    return new_data

In [ ]:
loaded_y_scaler = joblib.load('inverse_pipeline.pkl')

In [36]:
loaded_model = joblib.load('gb_california_model.pkl')


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Example new data
new_data = pd.DataFrame({
    'longitude': [-118.40],
    'latitude': [34.09],
    'total_rooms': [10000.0],
    'total_bedrooms': [1000.0],
    'population': [3000.0],
    'households': [1000.0],
    'median_income': [15.0],
    'housing_median_age': [5.0],
    'ocean_proximity': ['NEAR OCEAN']
})



# Predict on the test data using the loaded model
new_data = preprocess_new_data(new_data)
y_test_pred_loaded = loaded_model.predict(new_data).reshape(-1, 1)

scaled = scaler.fit_transform(y_test_pred_loaded)

y_test_pred_loaded_original = scaled.inverse_transform(y_test_pred_loaded).flatten()

y_test_pred_loaded_original

C:\Users\Hadi\AppData\Local\Temp\ipykernel_4208\3014033953.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_data['ocean_proximity'] = new_data['ocean_proximity'].replace({'ISLAND': 0, 'NEAR OCEAN': 1, '<1H OCEAN': 2, 'NEAR BAY': 3, 'INLAND': 4})
c:\Users\Hadi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'inverse_transform'

In [30]:
new_data

,longitude,latitude,total_rooms,total_bedrooms,population,households,median_income,housing_median_age,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-118.4,34.09,10000.0,1000.0,3000.0,1000.0,15.0,5.0,1,10.0,0.1,3.0


In [28]:
y_test_pred_loaded

array([[1.47782614]])

In [ ]:
# extremely low value data test
cheap_house = pd.DataFrame({
    'longitude': [-118.40],
    'latitude': [34.09],
    'total_rooms': [10000.0],
    'total_bedrooms': [1000.0],
    'population': [3000.0],
    'households': [1000.0],
    'median_income': [15.0],
    'housing_median_age': [5.0],
    'ocean_proximity': ['ISLAND']
})

# Predict on the test data using the loaded model
cheap_house = preprocess_new_data(cheap_house)
y_test_pred_loaded = loaded_model.predict(cheap_house)
y_test_pred_loaded_original = loaded_y_scaler.inverse_transform(y_test_pred_loaded.reshape(-1, 1)).flatten()

y_test_pred_loaded_original

In [ ]:
expensive_house = pd.DataFrame({
    'longitude': [-118.40],
    'latitude': [34.09],
    'total_rooms': [10000.0],
    'total_bedrooms': [1000.0],
    'population': [3000.0],
    'households': [1000.0],
    'median_income': [15.0],
    'housing_median_age': [5.0],
    'ocean_proximity': ['INLAND']
})


# Predict on the test data using the loaded model
expensive_house = preprocess_new_data(expensive_house)
y_test_pred_loaded = loaded_model.predict(expensive_house)
y_test_pred_loaded_original = loaded_y_scaler.inverse_transform(y_test_pred_loaded.reshape(-1, 1)).flatten()

y_test_pred_loaded_original

In [ ]:
batch_test

In [ ]:
# Predicting New Data Batch

# Load the dataset
data_path = "batch_test.csv"
batch_test = pd.read_csv(data_path)

batch_test = preprocess_new_data(batch_test)
y_pred = loaded_model.predict(batch_test)
y_pred_origin = loaded_y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_pred_origin.shape

In [ ]:
batch_test.head()

In [ ]:
batch_test['median_house_value']=y_pred_origin
batch_test.head()

In [ ]:
from google.cloud import bigquery

table_id = 'batch_test_predicted2'

# Construct a bigquery client object
client = bigquery.Client()

# define the full table id
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

batch_test.columns = ['_'.join(i.split(' ')) for i in batch_test.columns]

job = client.load_table_from_dataframe(batch_test,table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

In [ ]:
# # loaded_model.predict(X_test.iloc[3:13])
# y_pred_file_cloud = loaded_model.predict(auto_cloud)
# y_pred_file_cloud[:13]

In [ ]:
# auto_cloud['CLV Prediction'] = y_pred_file_cloud
# auto_cloud

---
<center>Thank You</center>

---